In [ ]:
from google.colab import files
uploaded = files.upload()

# 1. Combine 21 files thành 1 file master data
# Chỉ lấy những files có pattern "us-shein-"
import pandas as pd
import glob
csv_files = glob.glob("/content/us-shein-*.csv")

# Tạo list rỗng để chứa các dataframe của từng files
dfs = []
# Đọc file, thêm từng df vào dfs + lấy tên cate từ tên file & thêm cột cate vào dfs này luôn
for f in csv_files:
  df = pd.read_csv(f, low_memory = False)
  category = f.split('us-shein-')[1].rsplit('-', 1)[0]
  df['category'] = category
  dfs.append(df)
# Ghép cột
final_df = pd.concat(dfs, axis= 0, join= 'outer', ignore_index=True)

# Export data full (đã export master raw data này về & xử lý 1 xíu nên thêm # để python ngưng export data nữa)
## final_df.to_csv('shein_master_raw_data.csv', index= False)
## files.download('shein_master_raw_data.csv')

Saving shein_master_raw_data.csv to shein_master_raw_data (1).csv


In [4]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
# 2. Check %blank, %error, %zero value, outlier, dtypes
df = pd.read_csv('shein_master_raw_data.csv')

# Scan chất lượng dữ liệu
report =[]
for col in df.columns:
  col_data = df[col]
## Check độ dài cột
  n = len(col_data)
## Check %blank
  blank_pct = col_data.isna().mean()*100
## Check Dtype
  dtype = col_data.dtype
## Khởi tạo mặc định (dùng None thay vì dùng số 0 vì mình đang rà cho cả cột numeric & category. Mà chỉ cột numeric mới tính toán mấy chỉ số này được, nên để vậy để nó hiện kết quả None đối với những cột Category, hiện 0 thì dễ hiểu nhầm)
  error_pct = None
  zero_pct = None
  min_val = max_val = None
  outlier_pct = None
## Check numeric convert để tính %error
  if dtype == "object":
    numeric_try = pd.to_numeric(col_data, errors="coerce")
    error_pct = numeric_try.isna().mean()*100
## Check %zero
  if pd.api.types.is_numeric_dtype(col_data):
    zero_pct = (col_data == 0).mean()*100
## Check outliers
    min_val = col_data.min()
    max_val = col_data.max()
    Q1 = col_data.quantile(0.25)
    Q3 = col_data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - IQR * 1.5
    upper_bound = Q3 + IQR * 1.5
    outlier_pct = ((col_data < lower_bound) | (col_data > upper_bound)).mean()*100
 ## Tạo report tổng hợp
  report.append({
      "column": col,
      "dtype": str(dtype),
      "%blank" : round(blank_pct, 2),
      "%zero": round(zero_pct, 2) if zero_pct is not None else None,
      "min": min_val, "max": max_val,
      "%outlier": round(outlier_pct, 2) if outlier_pct is not None else None,
      "%error": round(error_pct, 2) if error_pct is not None else None})
## Sau vòng for
scan_report = pd.DataFrame(report)
scan_report

# Xử lý sau khi check
## rank-title: Giữ duy nhất giá trị "Best seller"
df['rank-title'] = df['rank-title'].apply(lambda x: "Best Sellers" if pd.notna(x) and 'best sellers' in str(x).lower()  else None)
## price: bỏ dấu $, chuyển về dtype numeric
df['price'] = df['price'].replace({r'[\$,]':''}, regex=True)
df['price'] = pd.to_numeric(df['price'], errors= 'coerce')
## discount: Chuyển sang dạng số
df['discount'] = df['discount'].str.replace('%', '', regex=False)
df['discount'] = pd.to_numeric(df['discount'], errors= 'coerce')
df.loc[df['discount'] > 100, 'discount'] = None
## selling_proposition: giữ lại duy nhất giá trị "sold recently"
df['selling_proposition'] = df['selling_proposition'].apply(lambda x: "sold recently" if pd.notna(x) and "sold recently" in str(x).lower() else None)
## color-count: điền giá trị trống bằng "Unknown"
df['color-count'] = df['color-count'].fillna('Unknown').astype(str)

# Export cleaned data
df.to_csv('shein_master_cleaned_data.csv', index= False)
files.download('shein_master_cleaned_data.csv')

Saving shein_master_raw_data.csv to shein_master_raw_data (3).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>